In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# from metrics import MAE, MAPE, RMSE

plt.style.use('fivethirtyeight')


In [ ]:
'''npz文件读取'''
def get_dataflow(dpath="./data/PEMS08/PEMS08.npz"):
    flow_data = np.load(dpath,allow_pickle=True)
    return flow_data['data']
# poem=np.load(npz_path,allow_pickle=True)
# poem.files
# data = poem['data']
# print(data.shape)

In [ ]:
'''数据集读取与可视化'''
traffic_data = get_dataflow()
print("data size {}".format(traffic_data.shape))
# 采样某个节点的数据
node_id = 2
# plt.plot(traffic_data[: 24 * 12, node_id, 0], label="flow")
# plt.plot(traffic_data[: 24 * 12, node_id, 1], label="speed")
# plt.plot(traffic_data[: 24 * 12, node_id, 2], label="occupancy")
plt.plot(traffic_data[:, node_id, 0], label="flow")
plt.plot(traffic_data[:, node_id, 1], label="speed")
plt.plot(traffic_data[:, node_id, 2], label="occupancy")
plt.legend(loc=0)
plt.savefig("./assets/vis.png")
plt.show()

In [ ]:
import numpy as np
from scipy.optimize import linprog

# 同构线性规划求解节点之间的 Wasserstein 距离
def wasserstein_distance(p, q, D):
    A_eq = []
    for i in range(len(p)):
        A = np.zeros_like(D)
        A[i, :] = 1
        A_eq.append(A.reshape(-1))
    for i in range(len(q)):
        A = np.zeros_like(D)
        A[:, i] = 1
        A_eq.append(A.reshape(-1))
    A_eq = np.array(A_eq)
    # 将分布 p 和分布 q 的元素合并成一个向量，作为等式约束的右侧向量。
    b_eq = np.concatenate([p, q])
    D = np.array(D)
    # 将矩阵 D 扁平化为一维数组
    D = D.reshape(-1)
    # 解决一个线性规划问题，其中目标函数系数为 D，
    #   等式约束系数矩阵为 A_eq，等式约束右侧向量为 b_eq，并保存求解结果
    result = linprog(D, A_eq=A_eq[:-1], b_eq=b_eq[:-1])
    #  从线性规划结果中获取目标函数的最小值，即最小运输成本1
    myresult = result.fun
    return myresult

def spatial_temporal_aware_distance(x, y):
    x, y = np.array(x), np.array(y)
    # L2范数
    x_norm = (x**2).sum(axis=1, keepdims=True)**0.5
    y_norm = (y**2).sum(axis=1, keepdims=True)**0.5
    print('x_norm: ', x_norm.shape)
    print('y_norm: ', y_norm.shape)
    # x y _norm (35, 1) 对应35天的每天的流量数据的L2范数

    # p->
    p = x_norm[:, 0] / x_norm.sum()
    print('p： ', p.shape)
    q = y_norm[:, 0] / y_norm.sum()
    new_x = x/x_norm
    print('new_x: ', new_x.shape)
    D = 1 - np.dot(x / x_norm, (y / y_norm).T)
    return wasserstein_distance(p, q, D)